In [1]:
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql import Window
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1663742679722_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
acc = sc.accumulator(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
window = Window.partitionBy('User_id').orderBy('Time_Stamp')

def somefunc(value,id):
  global acc

  if   value == '0' : 
    acc = 1
    return('Session_id '+str(acc)+' of '+id)

  elif value > 30:
    acc+=1
    return('Session_id '+str(acc)+' of '+id)
  else:
    return('Session_id '+str(acc)+' of '+id)
        
udfsomefunc = udf(somefunc, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df = spark.read.csv(
    "s3://aws-emr-resources-971996090633-ap-south-1/notebooks/e-DWW3JFF2I6LPKWT6PH45NCHKV/Tech_talk_Data.csv",
    header=True,
    inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df.createOrReplaceTempView("data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+--------------+
|          User_id|  URL|    Time_Stamp|
+-----------------+-----+--------------+
|The Giant Company|page1|10/29/22 11:10|
|       Giant Food|page1|10/29/22 11:10|
|      Stop & Shop|page1|10/29/22 11:10|
+-----------------+-----+--------------+
only showing top 3 rows

In [7]:
data_min = spark.sql(""" 
select User_id, Time_stamp,
case when diff is null then 0
else (unix_timestamp(to_timestamp(Time_Stamp,"dd/mm/yy HH:mm"))- unix_timestamp(diff))/60 end as session_difference
from (select User_id, URL, Time_Stamp, 
lag(to_timestamp(Time_Stamp,"dd/mm/yy HH:mm"),1) over(partition by User_id order by unix_timestamp(to_timestamp(Time_Stamp,"dd/mm/yy HH:mm")) asc) as diff
 from data)""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
data_min.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------+------------------+
|    User_id|    Time_stamp|session_difference|
+-----------+--------------+------------------+
|Stop & Shop| 10/29/22 0:11|               0.0|
|Stop & Shop| 10/29/22 0:41|              30.0|
|Stop & Shop|10/29/22 11:10|             629.0|
|Stop & Shop|10/31/22 11:18|               8.0|
+-----------+--------------+------------------+
only showing top 4 rows

In [11]:
df3 = data_min.withColumn("Session", udfsomefunc("session_difference","User_id"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'udfsomefunc' is not defined
Traceback (most recent call last):
NameError: name 'udfsomefunc' is not defined



In [16]:
# data_min = spark.sql(""" select User_id,Time_Stamp,case when diff is null then 0
# else datediff(diff,Time_stamp) end as session from (select User_id, URL, Time_Stamp, lag(Time_Stamp,1) over(partition by User_id order by Time_Stamp) as diff from data)"""
#                     )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
data_min.sort(col('User_id'), col('Time_Stamp')).show(12)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------+------------------+
|   User_id|    Time_stamp|session_difference|
+----------+--------------+------------------+
|    GF/TGC| 10/29/22 0:11|               0.0|
|    GF/TGC| 10/29/22 0:41|              30.0|
|    GF/TGC|10/29/22 11:10|             629.0|
|    GF/TGC| 11/2/22 11:18|            1448.0|
|Giant Food| 10/29/22 0:11|               0.0|
|Giant Food| 10/29/22 0:41|              30.0|
|Giant Food|10/29/22 11:10|             629.0|
|Giant Food|10/30/22 11:18|               8.0|
|   S&S/TGC| 10/29/22 0:11|               0.0|
|   S&S/TGC| 10/29/22 0:41|              30.0|
|   S&S/TGC|10/29/22 11:10|             629.0|
|   S&S/TGC| 11/1/22 11:18|            1448.0|
+----------+--------------+------------------+
only showing top 12 rows

In [32]:
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df2 = data_min.withColumn(
    'DiffInSeconds',
    col("Time_Stamp").cast("long") - col('diff').cast("long"))
df2.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+--------------+--------------+-------------+
|User_id          |URL  |Time_Stamp    |diff          |DiffInSeconds|
+-----------------+-----+--------------+--------------+-------------+
|Stop & Shop      |page4|10/29/22 0:11 |null          |null         |
|Stop & Shop      |page3|10/29/22 0:41 |10/29/22 0:11 |null         |
|Stop & Shop      |page1|10/29/22 11:10|10/29/22 0:41 |null         |
|Stop & Shop      |page2|10/31/22 11:18|10/29/22 11:10|null         |
|GF/TGC           |page4|10/29/22 0:11 |null          |null         |
|GF/TGC           |page3|10/29/22 0:41 |10/29/22 0:11 |null         |
|GF/TGC           |page1|10/29/22 11:10|10/29/22 0:41 |null         |
|GF/TGC           |page2|11/2/22 11:18 |10/29/22 11:10|null         |
|Giant Food       |page4|10/29/22 0:11 |null          |null         |
|Giant Food       |page3|10/29/22 0:41 |10/29/22 0:11 |null         |
|Giant Food       |page1|10/29/22 11:10|10/29/22 0:41 |null         |
|Giant Food       |p

In [19]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- User_id: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Time_Stamp: string (nullable = true)

In [3]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- User_id: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Time_Stamp: string (nullable = true)

In [73]:
schema = StructType([
    StructField("User_id", StringType(), True),
    StructField("URL", IntegerType(), False),
    StructField("Time_Stamp", TimestampType(), False)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
df.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(User_id='The Giant Company', URL='page1', Time_Stamp='10/29/2022 11:10'), Row(User_id='Giant Food', URL='page1', Time_Stamp='10/29/2022 11:10'), Row(User_id='Stop & Shop', URL='page1', Time_Stamp='10/29/2022 11:10'), Row(User_id='S&S/TGC', URL='page1', Time_Stamp='10/29/2022 11:10'), Row(User_id='GF/TGC', URL='page1', Time_Stamp='10/29/2022 11:10'), Row(User_id='The Giant Company', URL='page4', Time_Stamp='10/29/2022 0:11'), Row(User_id='Giant Food', URL='page4', Time_Stamp='10/29/2022 0:11'), Row(User_id='Stop & Shop', URL='page4', Time_Stamp='10/29/2022 0:11'), Row(User_id='S&S/TGC', URL='page4', Time_Stamp='10/29/2022 0:11'), Row(User_id='GF/TGC', URL='page4', Time_Stamp='10/29/2022 0:11'), Row(User_id='The Giant Company', URL='page3', Time_Stamp='10/29/2022  15:11:23 AM'), Row(User_id='Giant Food', URL='page3', Time_Stamp='10/29/2022  15:11:23 AM'), Row(User_id='Stop & Shop', URL='page3', Time_Stamp='10/29/2022  15:11:23 AM'), Row(User_id='S&S/TGC', URL='page3', Time_Stamp='10

In [71]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- User_id: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Time_Stamp: string (nullable = true)

In [56]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [57]:
acc = sc.accumulator(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
window = Window.partitionBy('User_id').orderBy('Time_Stamp')


def somefunc(value, id):
    global acc

    if value is None:
        acc = 1
        return ('Session_id ' + str(acc) + ' of ' + id)

    elif value > 30:
        acc += 1
        return ('Session_id ' + str(acc) + ' of ' + id)
    else:
        return ('Session_id ' + str(acc) + ' of ' + id)


udfsomefunc = F.udf(somefunc, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
df2 = df.withColumn("unix_timestamp", unix_timestamp(F.col("Time_Stamp")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
df2.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+----------------+--------------+
|          User_id|  URL|      Time_Stamp|unix_timestamp|
+-----------------+-----+----------------+--------------+
|The Giant Company|page1|10/29/2022 11:10|          null|
|       Giant Food|page1|10/29/2022 11:10|          null|
|      Stop & Shop|page1|10/29/2022 11:10|          null|
+-----------------+-----+----------------+--------------+
only showing top 3 rows

In [65]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+--------------------+
|          User_id|  URL|          Time_Stamp|
+-----------------+-----+--------------------+
|The Giant Company|page1|    10/29/2022 11:10|
|       Giant Food|page1|    10/29/2022 11:10|
|      Stop & Shop|page1|    10/29/2022 11:10|
|          S&S/TGC|page1|    10/29/2022 11:10|
|           GF/TGC|page1|    10/29/2022 11:10|
|The Giant Company|page4|     10/29/2022 0:11|
|       Giant Food|page4|     10/29/2022 0:11|
|      Stop & Shop|page4|     10/29/2022 0:11|
|          S&S/TGC|page4|     10/29/2022 0:11|
|           GF/TGC|page4|     10/29/2022 0:11|
|The Giant Company|page3|10/29/2022  15:11...|
|       Giant Food|page3|10/29/2022  15:11...|
|      Stop & Shop|page3|10/29/2022  15:11...|
|          S&S/TGC|page3|10/29/2022  15:11...|
|           GF/TGC|page3|10/29/2022  15:11...|
|The Giant Company|page2|    10/29/2022 11:18|
|       Giant Food|page2|    10/30/2022 11:18|
|      Stop & Shop|page2|    10/31/2022 11:18|
|          S&

In [53]:
df3 = df2.withColumn(
    'minutes',
    ((df.unix_timestamp - funcs.lag(df.unix_timestamp, 1).over(window)) /
     60).cast(IntegerType())).withColumn("Session",
                                         udfsomefunc("minutes", "SWID"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DataFrame' object has no attribute 'unix_timestamp'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 1307, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
AttributeError: 'DataFrame' object has no attribute 'unix_timestamp'



In [49]:
import pyspark.sql.functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
df3 = df2.withColumn("lag", F.lag(F.unix_timestamp, 1).over(window))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid argument, not a string or column: <function unix_timestamp at 0x7fb9f6269ef0> of type <class 'function'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 825, in lag
    return Column(sc._jvm.functions.lag(_to_java_column(col), count, default))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/column.py", line 53, in _to_java_column
    "function.".format(col, type(col)))
TypeError: Invalid argument, not a string or column: <function unix_timestamp at 0x7fb9f6269ef0> of type <class 'function'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

